In [1]:
import string

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim



In [2]:
data = pd.read_csv("./IMDB Dataset.csv")

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [5]:
data.shape

(50000, 2)

In [6]:
def label_encode(lb):
    if lb == "positive":
        return 1
    else:
        return 0

In [7]:
data["sentiment_labels"] = data["sentiment"].apply(label_encode) 

In [8]:
data.head()

,review,sentiment,sentiment_labels
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [9]:
data["sentiment_labels"].value_counts()

sentiment_labels
1    25000
0    25000
Name: count, dtype: int64

In [10]:
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [11]:
def get_vocab(data):
    reviews_txt = ""
    for i in range(data.shape[0]):
        reviews_txt += f" {data.review[i]}"
    return reviews_txt

In [12]:
vocab = get_vocab(data)

In [13]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data.review)

In [14]:
X.shape

(50000, 101895)

In [15]:
x = X.toarray

In [16]:
small_df = pd.DataFrame(X[:10].toarray())
small_df

,0,1,2,3,4,5,6,7,8,9,...,101885,101886,101887,101888,101889,101890,101891,101892,101893,101894
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X.shape

(50000, 101895)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, data["sentiment_labels"], test_size=0.2, random_state=42)

In [19]:
lin_svc = LinearSVC()

In [20]:
lin_svc.fit(X_train, y_train)

/home/mpho/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [21]:
y_preds = lin_svc.predict(X_test)

In [22]:
len(y_preds)

10000

In [23]:
print(accuracy_score(y_test, y_preds))

0.8666


In [24]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87      4961
           1       0.87      0.87      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [25]:
cat_n = MultinomialNB()
cat_n.fit(X_train, y_train)

MultinomialNB()

In [26]:
y_preds = cat_n.predict(X_test)
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      4961
           1       0.87      0.82      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [27]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/home/mpho/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [28]:
y_preds = log_reg.predict(X_test)

In [29]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.89      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [30]:
def remove_punct(content):
    clean_text = content.translate(str.maketrans('', '', string.punctuation))
    return content

In [31]:
content = remove_punct(vocab)


In [32]:
content = content.lower()

In [33]:
print(content[:100])

 one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. th


In [34]:
content_arr = content.split(" ")

In [35]:
#construct input IDS
def get_input_ids(content):
    dict_ = {}
    x = 0
    for i, word in enumerate(content):
        if word in dict_:
            pass
        elif word != "" and word not in dict_:
            dict_[word] = x
            x+=1
    return dict_

In [36]:
input_ids = get_input_ids(content_arr)

In [37]:
vocab_size = len(input_ids)
vocab_size

392053

In [38]:
max(input_ids.values())

392052

In [39]:
def remove_punct_from_df(review):
    clean_text = review.translate(str.maketrans('', '', string.punctuation))
    return clean_text

In [40]:
len(input_ids)

392053

In [41]:
data["n_review"] = data["review"].apply(remove_punct_from_df)

In [42]:
data.head()

,review,sentiment,sentiment_labels,n_review
0,One of the other reviewers has mentioned that ...,positive,1,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,1,A wonderful little production br br The filmin...
2,I thought this was a wonderful way to spend ti...,positive,1,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,0,Basically theres a family where a little boy J...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,Petter Matteis Love in the Time of Money is a ...


In [43]:
def convert_to_input_ids(review):
    split_review = review.split(" ")
    id_arr = []
    for r in split_review:
        if r in input_ids:
            id_arr.append(input_ids[r])
    return id_arr
    

In [44]:
data["input_ids"] = data["n_review"].apply(convert_to_input_ids)

In [45]:
data.head()

,review,sentiment,sentiment_labels,n_review,input_ids
0,One of the other reviewers has mentioned that ...,positive,1,One of the other reviewers has mentioned that ...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 39..."
1,A wonderful little production. <br /><br />The...,positive,1,A wonderful little production br br The filmin...,"[200, 201, 246, 153180, 153180, 204, 205, 22, ..."
2,I thought this was a wonderful way to spend ti...,positive,1,I thought this was a wonderful way to spend ti...,"[285, 21, 35, 52, 200, 286, 63, 287, 288, 83, ..."
3,Basically there's a family where a little boy ...,negative,0,Basically theres a family where a little boy J...,"[29681, 52, 371, 90, 52, 201, 372, 374, 29681,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,Petter Matteis Love in the Time of Money is a ...,"[44, 2, 1, 22, 52, 429, 430, 394, 63, 1489, 43..."


In [46]:
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)

In [47]:
train_df, valid_df = train_test_split(train_df, test_size=0.05, random_state=42)

In [48]:
valid_df.shape, train_df.shape

((1750, 5), (33250, 5))

In [49]:
train_df.head()

,review,sentiment,sentiment_labels,n_review,input_ids
32059,Went to the premiere at the Tribeca Film Festi...,positive,1,Went to the premiere at the Tribeca Film Festi...,"[63, 2, 10457, 335, 2, 44, 38, 1485, 847, 2, 3..."
47328,God bless Randy Quaid...his leachorous Cousin ...,negative,0,God bless Randy Quaidhis leachorous Cousin Edd...,"[18504, 379197, 44, 38, 6670, 12835, 2, 53, 75..."
18933,This is absolutely the most stupidest movie ev...,negative,0,This is absolutely the most stupidest movie ev...,"[22, 1485, 2, 332, 9112, 381, 146, 4853, 44, 2..."
3234,You're waiting to see if the remake is better ...,positive,1,Youre waiting to see if the remake is better o...,"[2697, 63, 232, 194, 2, 4592, 22, 795, 57, 154..."
15741,"I found Grey Garden's to be a gripping film, a...",positive,1,I found Grey Gardens to be a gripping film an ...,"[3887, 63, 15, 52, 8556, 394, 86, 12113, 15318..."


In [50]:
train_df.shape

(33250, 5)

In [51]:
test_df.shape

(15000, 5)

In [52]:
#data loading
class TextDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        input_ids = self.df.iloc[idx]['input_ids']   
        label = self.df.iloc[idx]['sentiment_labels']         
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(label, dtype=torch.long)

In [55]:
def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch])

    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    return padded_input_ids, labels

dataset = TextDataset(train_df)  
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [56]:
valid_dataset = TextDataset(valid_df)  
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [57]:
dataloader.dataset[2]

(tensor([    22,   1485,      2,    332,   9112,    381,    146,   4853,     44,
           2959,      1,     52,   3046,   4910,    619,     35, 173840,   1304,
             63,   2821,     21,   2426,      1,  22268,     93,    539,    311,
            666,    822,     44,    524,    288,     21,   1502,      2,  26169,
             38,    323,   4910,    173,   1548,    478,    666,     21,     31,
             35,    499,     35,     52,     35,      2,   3046,    609,    575,
             63,     15,   1917,    236,     52,   3688,    838,    751,   4580,
             93,    120,    539,     52,    381,   1003,     98,    879,     63,
            367,   5771,      2,    939,    381,   1120,     93,    342,   1263,
             22,    725,    259,   7677,   1011,   1772,     21,   3192,    575,
            311,   7971,    288,     83,    250,   2257,   3688,     86,   7971,
           4106,   9854,     44,    250,  61560,     38,   3932,      0,   3958,
            367,    393,    

In [59]:
class SentimentClassifier(nn.Module):
    def __init__(self, num_embeddings=vocab_size, dim_embeddings=100):
        super(SentimentClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings, dim_embeddings)
        self.block = nn.Sequential(
            nn.Conv1d(dim_embeddings, 156, 3, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv1d(156, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(128, 112, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(112, 64, 3, padding=1),
            nn.ReLU(inplace=True)  
        )
        self.global_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(64, 2)
        
    def forward(self, x):
        x = self.embedding_layer(x)      
        x = x.permute(0, 2, 1)           
        x = self.block(x)                
        x = self.global_pool(x).squeeze(-1)  
        out = self.fc1(x)                
        return out

In [86]:
model = SentimentClassifier()

In [87]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [88]:
model.to(device)

SentimentClassifier(
  (embedding_layer): Embedding(392053, 100)
  (block): Sequential(
    (0): Conv1d(100, 156, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): Conv1d(156, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU(inplace=True)
    (4): Conv1d(128, 112, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): ReLU(inplace=True)
    (6): Conv1d(112, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU(inplace=True)
  )
  (global_pool): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=64, out_features=2, bias=True)
)

In [89]:
optimizer = optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

In [91]:
def train(epochs=50):
    model.train() 
    patience = 0
    prev_loss = 0
    for epoch in range(epochs):
        epoch_loss = 0
        valid_loss = 0
        
        for reviews, labels in dataloader:
            reviews = reviews.to(device)
            labels = labels.long().to(device)

            optimizer.zero_grad()
            output = model(reviews) 
            loss = criterion(output, labels)  
            loss.backward()  
            optimizer.step()            

            epoch_loss += loss.item()
        
        # Validation
        model.eval()
        with torch.no_grad():
            for v_review, lb in valid_loader:
                v_review = v_review.to(device)
                lb = lb.long().to(device)
                preds = model(v_review)
                v_loss = criterion(preds, lb)
                valid_loss += v_loss.item()
        model.train()

        avg_loss = epoch_loss / len(dataloader)
        avg_v_loss = valid_loss / len(valid_loader)

        if epoch == 0:
            prev_loss = avg_v_loss
    
        if avg_v_loss > prev_loss:
            patience+=1
        elif avg_v_loss < prev_loss:
            torch.save(model.state_dict(), f"best_model.pth")
            patience = 0
            prev_loss = avg_v_loss
            
            
        
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Validation Loss: {avg_v_loss:.4f}")
        if patience == 5:
            break


In [92]:
train()

Epoch 1, Loss: 0.6931, Validation Loss: 0.6929
Epoch 2, Loss: 0.6922, Validation Loss: 0.6927
Epoch 3, Loss: 0.6911, Validation Loss: 0.6900
Epoch 4, Loss: 0.6881, Validation Loss: 0.6862
Epoch 5, Loss: 0.6708, Validation Loss: 0.6310
Epoch 6, Loss: 0.6176, Validation Loss: 0.7047
Epoch 7, Loss: 0.5831, Validation Loss: 0.6461
Epoch 8, Loss: 0.5455, Validation Loss: 0.5273
Epoch 9, Loss: 0.5040, Validation Loss: 0.6997
Epoch 10, Loss: 0.4683, Validation Loss: 0.4426
Epoch 11, Loss: 0.4309, Validation Loss: 0.5988
Epoch 12, Loss: 0.4006, Validation Loss: 1.0704
Epoch 13, Loss: 0.3763, Validation Loss: 0.6003
Epoch 14, Loss: 0.3469, Validation Loss: 0.4073
Epoch 15, Loss: 0.3226, Validation Loss: 0.7261
Epoch 16, Loss: 0.2983, Validation Loss: 0.6121
Epoch 17, Loss: 0.2712, Validation Loss: 0.6320
Epoch 18, Loss: 0.2474, Validation Loss: 0.4151
Epoch 19, Loss: 0.2228, Validation Loss: 0.4304


In [93]:
model = SentimentClassifier()
model.load_state_dict(torch.load("./best_model.pth", weights_only=True))

<All keys matched successfully>

In [94]:
model.to(device)

SentimentClassifier(
  (embedding_layer): Embedding(392053, 100)
  (block): Sequential(
    (0): Conv1d(100, 156, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): Conv1d(156, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU(inplace=True)
    (4): Conv1d(128, 112, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): ReLU(inplace=True)
    (6): Conv1d(112, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU(inplace=True)
  )
  (global_pool): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=64, out_features=2, bias=True)
)

In [95]:
test_dataset = TextDataset(test_df)  # your dataframe
test_loader = DataLoader(test_dataset,batch_size=16, shuffle=True, collate_fn=collate_fn)

In [96]:
model.eval() 
total_correct = 0
total_samples = 0
pred_labels = []
true_labels = []

with torch.no_grad():
    for dt, labels in test_loader:
        dt = dt.to(device)
        labels = labels.to(device)
        true_labels.append(labels)
        output = model(dt)
        
        predictions = torch.argmax(output, dim=1)
        pred_labels.append(predictions)
        
        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)



accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy:.4f}")
        

Test Accuracy: 0.8091


In [97]:
pred_labels

[tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1], device='cuda:0'),
 tensor([1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1], device='cuda:0'),
 tensor([0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], device='cuda:0'),
 tensor([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1], device='cuda:0'),
 tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1], device='cuda:0'),
 tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1], device='cuda:0'),
 tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0], device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0], device='cuda:0'),
 tensor([0, 

In [98]:
true_labels

[tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1], device='cuda:0'),
 tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1], device='cuda:0'),
 tensor([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1], device='cuda:0'),
 tensor([0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], device='cuda:0'),
 tensor([0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1], device='cuda:0'),
 tensor([0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], device='cuda:0'),
 tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0], device='cuda:0'),
 tensor([1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0], device='cuda:0'),
 tensor([0, 

In [99]:
y_preds = []
for lb in pred_labels:
    for l in lb:
        y_preds.append(l.item())

In [100]:
y_preds

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,


In [101]:
t_labels = []
for lb in true_labels:
    for l in lb:
        t_labels.append(l.item())

In [102]:
print(classification_report(t_labels, y_preds))

              precision    recall  f1-score   support

           0       0.78      0.85      0.81      7411
           1       0.84      0.77      0.80      7589

    accuracy                           0.81     15000
   macro avg       0.81      0.81      0.81     15000
weighted avg       0.81      0.81      0.81     15000



## Recurrent neural networks

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.block = nn.Sequential()